In [4]:
import sqlite3
import pandas as pd

# Replace 'your_database_name.db' with the desired name for your SQLite database
database_name = 'test_prices.db'
csv_file_path = 'DataPrice-2024-01-29.csv'

# Create a connection to the SQLite database
conn = sqlite3.connect(database_name)

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Elimina las columnas especificadas
#columns_to_drop = ['Comarca', 'id', 'data_source', 'Cantidad (t)', 'Variedad', 'Familia']
#df = df.drop(columns=columns_to_drop)

# Formatea la columna de fecha sin la hora
df['Fecha'] = pd.to_datetime(df['Fecha']).dt.date

# Write the DataFrame to the SQLite database
df.to_sql('precio_del_cultivo', conn, index=False, if_exists='replace')

# Commit the changes and close the connection
conn.commit()
conn.close()

print(f"Database '{database_name}' created and filled with data from '{csv_file_path}'.")


Database 'test_prices.db' created and filled with data from 'DataPrice-2024-01-29.csv'.


/var/folders/13/vtwv0p951k11cf7tvhd750jw0000gn/T/ipykernel_16729/1164132424.py:22: UserWarning: Parsing dates in %d-%m-%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Fecha'] = pd.to_datetime(df['Fecha']).dt.date


In [5]:
import sqlite3

# Conéctate a la base de datos
conn = sqlite3.connect('test_prices.db')  # Reemplaza con el nombre real de tu base de datos
cursor = conn.cursor()

# Ejecuta una consulta SQL
cursor.execute('SELECT * FROM precio_del_cultivo')  # Reemplaza con el nombre real de tu tabla

results = cursor.fetchall()

# Imprime los resultados
for row in results:
    print(row)

# Cierra la conexión
conn.close()

('Maíz', 'Nacional', 'Cereales', '2022-10-18', 0.0, 350.75, 'León', 'Null', 121003, 31)
('Maíz', 'Nacional', 'Cereales', '2022-10-19', None, 348.75, 'León', None, 139010, 31)
('Maíz', 'Nacional', 'Cereales', '2022-10-20', None, 348.0, 'León', 'Castilla y León', 163016, 31)
('Maíz', 'Nacional', 'Cereales', '2022-10-20', 0.0, 350.0, 'León', None, 163856, 31)
('Maíz', 'Nacional', 'Cereales', '2022-10-25', None, 353.0, 'León', 'Castilla y León', 171047, 31)
('Maíz', 'Nacional', 'Cereales', '2022-10-26', None, 352.0, 'León', 'Castilla y León', 171055, 31)
('Maíz', 'Nacional', 'Cereales', '2022-10-28', 0.0, 350.0, 'León', None, 171302, 31)
('Maíz', 'Nacional', 'Cereales', '2022-10-31', None, 353.0, 'León', 'Castilla y León', 171309, 31)
('Maíz', 'Nacional', 'Cereales', '2022-11-02', None, 350.0, 'León', 'Castilla y León', 171603, 31)
('Maíz', 'Nacional', 'Cereales', '2022-11-03', None, 350.0, 'León', 'Castilla y León', 171618, 31)
('Maíz', 'Nacional', 'Cereales', '2022-11-05', None, 349.5, '

In [71]:
from langchain.sql_database import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///test_prices.db")


In [72]:
import api_key
import os
os.environ["OPENAI_API_KEY"] = api_key.OPENAI_API_KEY


In [73]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0,model_name='gpt-3.5-turbo')

In [74]:
from langchain_experimental.sql import SQLDatabaseChain
cadena = SQLDatabaseChain(llm = llm, database = db, verbose=False)

/Users/daniel/anaconda3/envs/abastores_gpt/lib/python3.11/site-packages/langchain_experimental/sql/base.py:76: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [78]:
# 5. Formato personalizado de respuesta


In [79]:
# 6. Función para hacer la consulta

def consulta(input_usuario):
    consulta = formato.format(question = input_usuario)
    resultado = cadena.run(consulta)
    return(resultado)

In [80]:
print(consulta('porque el precio del maiz bajo del 9 al 10 de enero del 2023?'))

En base a los datos obtenidos de la tabla "precio_del_cultivo", el precio del maíz bajó del 9 al 10 de enero del 2023 de 312.75 €/t a 310.5 €/t. Esta disminución en el precio podría estar relacionada con la noticia del 12 de enero que pronostica una baja en el precio de los cítricos debido a la sobreproducción, lo que podría estar afectando también al mercado de otros productos agrícolas como el maíz.


In [95]:
import requests
import pandas as pd

# Define the API endpoint
api_url = "https://api.eia.gov/v2/international/data/"

# Define the parameters
params = {
    "frequency": "monthly",
    "data[0]": "value",
    "facets[countryRegionId][]": "ESP",
    "start": "2000-01",
    "end": "2023-10",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": 0,
    "length": 5000,
    "api_key": "QPPZI67wpn6GRcwcQ5KdqEXGbxySLYsiXDZWLO7q",  # Replace with your actual API key
}

# Make the API request
response = requests.get(api_url, params=params)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Extract the relevant data from the JSON response (assuming data is in "series")
    series_data = data.get("response", [])

    # Convert the data to a DataFrame
    df = pd.DataFrame(series_data)

    # Explode the 'data' column
    df = df.join(json_normalize(df['data']).add_prefix('data_')).drop(columns='data')
    
    # Explode the 'description' column
    df = df.assign(description=df['description'].str.split('...').explode().reset_index(drop=True))
    
    # Display the resulting DataFrame
    print(df)
    # Display the DataFra
else:
    # Handle the error
    print(f"Error: {response.status_code} - {response.text}")


     total dateFormat frequency description data_period data_productId  \
0     2143    YYYY-MM   monthly                 2023-10              5   
1     2143    YYYY-MM   monthly                 2023-10             59   
2     2143    YYYY-MM   monthly                 2023-10             58   
3     2143    YYYY-MM   monthly                 2023-10             57   
4     2143    YYYY-MM   monthly                 2023-10             56   
...    ...        ...       ...         ...         ...            ...   
2138  2143    YYYY-MM   monthly                 2000-01              5   
2139  2143    YYYY-MM   monthly                 2000-01             53   
2140  2143    YYYY-MM   monthly                 2000-01             54   
2141  2143    YYYY-MM   monthly                 2000-01             55   
2142  2143    YYYY-MM   monthly                 2000-01             57   

                          data_productName data_activityId data_activityName  \
0              Petroleum and ot

In [100]:
df['data_productName'].value_counts()

data_productName
Petroleum and other liquids             286
Crude oil including lease condensate    286
Crude oil, NGPL, and other liquids      286
Refined petroleum products              286
Total petroleum and other liquids       286
NGPL                                    238
Refinery processing gain                238
Other liquids                           237
Name: count, dtype: int64

In [114]:
df.loc[df['data_productName']=='Crude oil including lease condensate']

,total,dateFormat,frequency,description,data_period,data_productId,data_productName,data_activityId,data_activityName,data_countryRegionId,data_countryRegionName,data_countryRegionTypeId,data_countryRegionTypeName,data_dataFlagId,data_dataFlagDescription,data_unitName,data_value,data_unit
3,2143,YYYY-MM,monthly,,2023-10,57,Crude oil including lease condensate,1,Production,ESP,Spain,c,Country,5,"A value exists, but rounds to zero; non-rounde...",thousand barrels per day,.012,TBPD
10,2143,YYYY-MM,monthly,,2023-09,57,Crude oil including lease condensate,1,Production,ESP,Spain,c,Country,None,None,thousand barrels per day,0,TBPD
18,2143,YYYY-MM,monthly,,2023-08,57,Crude oil including lease condensate,1,Production,ESP,Spain,c,Country,5,"A value exists, but rounds to zero; non-rounde...",thousand barrels per day,.02,TBPD
26,2143,YYYY-MM,monthly,,2023-07,57,Crude oil including lease condensate,1,Production,ESP,Spain,c,Country,5,"A value exists, but rounds to zero; non-rounde...",thousand barrels per day,.017,TBPD
34,2143,YYYY-MM,monthly,,2023-06,57,Crude oil including lease condensate,1,Production,ESP,Spain,c,Country,5,"A value exists, but rounds to zero; non-rounde...",thousand barrels per day,.015,TBPD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2119,2143,YYYY-MM,monthly,,2000-05,57,Crude oil including lease condensate,1,Production,ESP,Spain,c,Country,None,None,thousand barrels per day,4.548,TBPD
2124,2143,YYYY-MM,monthly,,2000-04,57,Crude oil including lease condensate,1,Production,ESP,Spain,c,Country,None,None,thousand barrels per day,4.7,TBPD
2129,2143,YYYY-MM,monthly,,2000-03,57,Crude oil including lease condensate,1,Production,ESP,Spain,c,Country,None,None,thousand barrels per day,4.806,TBPD
2134,2143,YYYY-MM,monthly,,2000-02,57,Crude oil including lease condensate,1,Production,ESP,Spain,c,Country,None,None,thousand barrels per day,4.862,TBPD


In [103]:
df.loc[df['data_productName']=='Petroleum and other liquids','data_value']

0       w
8       w
15      w
23      w
31      w
       ..
2118    w
2123    w
2128    w
2133    w
2138    w
Name: data_value, Length: 286, dtype: object